# PAC-UCB Algorithm

Einige Begründungen: Parameter 
q: Dieser Parameter steuert, wie stark die Exploration im Verhältnis zur Anzahl der bisherigen Ziehungen eines Arms zunimmt. Ein Wert q>1 bedeutet, dass die Exploration überproportional zur Anzahl der Ziehungen eines Arms zunimmt. Für eine moderate Exploration kann man q auf einen Wert zwischen 1.1 und 2 setzen. Um eine Vergleichbarkeit zu den vorherigen Algorithmen herzustellen und nicht zu aggressiv oder konservativ zu explorieren, wählen wir q=1.5

β: Dieser Parameter steht in Verbindung mit der Wahrscheinlichkeit, dass die obere Schranke eingehalten wird. Ein typischer Wert für β könnte in der Größenordnung von 0.1 oder 0.01 liegen. Wir wählen β=0.05, um eine gute Balance zwischen Exploration und Exploitation zu gewährleisten.

In [1]:
import numpy as np
import csv
import math

# Class BanditAlgorithm: Initialization

In [2]:
class BanditAlgorithm:
    def __init__(self, name):
        self.name = name
        self.results = []

    def add_result(self, param, iteration, total_reward, suboptimal_arms_count, total_regret, zeros_count, ones_count):
        self.results.append([param, iteration, total_reward, suboptimal_arms_count, round(total_regret, 2), zeros_count, ones_count])

    def save_results_to_csv(self, filename):
        with open(filename, mode='w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(['Timestep', 'Iteration', 'Total Reward', 'Suboptimal Arms Count', 'Total Regret', 'Zeros Count', 'Ones Count'])
            for result in self.results:
                writer.writerow(result)

    def calculate_average_results(self):
        avg_results = {}
        for result in self.results:
            param = result[0]
            if param not in avg_results:
                avg_results[param] = [0, 0, 0, 0, 0]
            avg_results[param][0] += result[2]  # Total Reward
            avg_results[param][1] += result[3]  # Suboptimal Arms Count
            avg_results[param][2] += result[4]  # Total Regret
            avg_results[param][3] += result[5]  # Zeros Count
            avg_results[param][4] += result[6]  # Ones Count
        
        for param in avg_results:
            avg_results[param] = [param] + [x / 100 for x in avg_results[param]]
        return list(avg_results.values())

## Definition PAC-UCB

In [3]:
def PAC_UCB_simulation(algorithm, arm_means, time_horizon, c=1, b=1, q=1.3, beta=0.05):
    num_arms = len(arm_means)
    rewards = np.zeros(num_arms)
    counts = np.zeros(num_arms)
    sum_of_squares = np.zeros(num_arms)
    total_reward = 0
    suboptimal_arms_count = 0
    total_regret = 0
    zeros_count = 0
    ones_count = 0

    for t in range(1, time_horizon + 1):
        if any(counts < 0):
            arm = np.argmin(counts)
        else:
            PAC = np.zeros(num_arms)
            for k in range(num_arms):
                if counts[k] > 0:
                    mean_reward = rewards[k] / counts[k]
                    exploration = max(np.log(num_arms * (counts[k] ** q) / beta), 2)
                    variance = (sum_of_squares[k] - counts[k] * (mean_reward ** 2)) / counts[k]
                    PAC[k] = mean_reward + np.sqrt((2 * variance * exploration) / counts[k]) + c * (3 * b * exploration / counts[k])
            arm = np.argmax(PAC)
        
        reward = np.random.binomial(1, arm_means[arm])
        counts[arm] += 1
        rewards[arm] += reward
        sum_of_squares[arm] += reward ** 2
        total_reward += reward

        if reward == 0:
            zeros_count += 1
        else:
            ones_count += 1

        if arm != np.argmax(arm_means):
            suboptimal_arms_count += 1
            total_regret += np.max(arm_means) - arm_means[arm]

    return total_reward, suboptimal_arms_count, total_regret, zeros_count, ones_count

## Run Simulation Function

In [4]:
def run_simulation(algorithm, parameters, arm_means):
    for iteration in range(1, 101):
        for param in parameters:
            total_reward, suboptimal_arms_count, total_regret, zeros_count, ones_count = PAC_UCB_simulation(algorithm, arm_means, param)
            algorithm.add_result(param, iteration, total_reward, suboptimal_arms_count, total_regret, zeros_count, ones_count)

## PAC-UCB for different time horizons

In [5]:
time_horizons = [2, 3, 100, 200, 2000, 10000, 20000, 40000, 60000, 80000, 100000]

# Beispiel-Algorithmen
algorithms = [
    BanditAlgorithm("7_PAC-UCB"),
]

# Beispiel-Mittelwerte der Arme
arm_means = np.array([0.495, 0.5])

# Simulation durchführen und Ergebnisse speichern
for algorithm in algorithms:
    run_simulation(algorithm, time_horizons, arm_means)
    results_path = r'C:/Users/canis/OneDrive\Dokumente/uni/uni-surface/FSS 2024/BA/bachelorarbeit_vrlfg/BA/github/BA_code/2_algorithms_results'
    algorithm.save_results_to_csv(f'{results_path}\{algorithm.name}_results_subopt_ver3.csv')
    avg_results = algorithm.calculate_average_results()
    with open(f'{results_path}\{algorithm.name}_average_results_subopt_ver3.csv', mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['Timestep', 'Average Total Reward', 'Average Suboptimal Arms', 'Average Regret', 'Average Zeros Count', 'Average Ones Count'])
        for result in avg_results:
            writer.writerow(result)